In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

In [2]:
chronom_train = pd.read_csv("../data/chronom_train.csv")
chronom_test = pd.read_csv("../data/chronom_test.csv")

In [5]:
chronom_train

,Unnamed: 0,NPLV,TYPE_OPER,NOP,VR_NACH,VR_KON,O2
0,35171,510008,межпл.прост.,межпл. простой,2011-01-01 10:28:13,2011-01-01 10:32:15,NaN
1,35162,510008,межпл.прост.,межпл. простой,2021-01-01 03:01:07,2021-01-01 03:08:11,NaN
2,35163,510008,межпл.прост.,Осмотр конвертера,2021-01-01 03:01:19,2021-01-01 03:03:43,NaN
3,35164,510008,межпл.прост.,Наведение гарнисажа,2021-01-01 03:03:43,2021-01-01 03:05:23,NaN
4,35165,510008,межпл.прост.,Слив шлака,2021-01-01 03:05:23,2021-01-01 03:06:09,NaN
...,...,...,...,...,...,...,...
34401,20943,512322,опер,Повалка,2021-04-26 18:31:26,2021-04-26 18:41:10,NaN
34402,20944,512322,опер,Выпуск металла,2021-04-26 18:41:10,2021-04-26 18:47:40,NaN
34403,20945,512322,опер,Отсечка шлака,2021-04-26 18:46:15,2021-04-26 18:46:15,NaN
34404,20946,512322,опер,Прерыв. выпуска,2021-04-26 18:47:40,2021-04-26 18:48:40,NaN


In [8]:
chronom_train["concat"] = chronom_train["TYPE_OPER"] + "_" + chronom_train["NOP"] 
chronom_test["concat"] = chronom_test["TYPE_OPER"] + "_" + chronom_test["NOP"] 

In [10]:
train_texts = chronom_train.groupby("NPLV")["concat"].apply(list)
test_texts = chronom_test.groupby("NPLV")["concat"].apply(list)

In [40]:
texts_for_w2v = pd.concat([train_texts, test_texts])

In [41]:
from gensim.models import Word2Vec

model_wv = Word2Vec(min_count=1, vector_size=20)

model_wv.build_vocab(list(texts_for_w2v), progress_per=200)

model_wv.train(texts_for_w2v, total_examples = model_wv.corpus_count, 
            epochs=10, report_delay=1)
model_wv.init_sims(replace=True)
print(model_wv)
# extract all vectors
# X = model_wv[model_wv.wv.vocab]
# X.shape

Word2Vec(vocab=49, vector_size=20, alpha=0.025)


<ipython-input-41-b2bf7cf73f2e>:9: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model_wv.init_sims(replace=True)


In [42]:
model_wv.wv[10]

array([ 0.10691107,  0.13116786, -0.09638966, -0.07551845,  0.28647608,
        0.2555186 , -0.33056185,  0.36623567, -0.2038137 ,  0.27680027,
       -0.11101172,  0.08156671,  0.2169744 ,  0.09303068,  0.21728037,
       -0.08753336,  0.4606937 ,  0.10487854, -0.30701485, -0.01939465],
      dtype=float32)

In [63]:
def tokens2m(tokens, model_wv):
    res = []
    for token in tokens:
        idx = model_wv.wv.key_to_index[token]
        res.append(model_wv.wv[idx])
        
    return np.mean(np.array(res), axis=0).tolist() + np.std(np.array(res), axis=0).tolist() + [np.linalg.norm(np.array(res))]

In [65]:
len(tokens2m(train_texts.iloc[0], model_wv))

41

In [39]:
model_wv.wv.key_to_index

{'межпл.прост._межпл. простой': 0,
 'опер_Нагрев лома': 1,
 'опер_Повалка': 2,
 'межпл.прост._Слив шлака': 3,
 'опер_Выпуск металла': 4,
 'опер_Завалка лома': 5,
 'опер_Заливка чугуна': 6,
 'опер_Продувка': 7,
 'опер_Слив шлака': 8,
 'межпл.прост._Наведение гарнисажа': 9,
 'опер_Прерыв. выпуска': 10,
 'межпл.прост._Осмотр конвертера': 11,
 'опер_Отсечка шлака': 12,
 'опер_Додувка на t': 13,
 'опер_Додувка на C': 14,
 'вн.пл.прост._Отсутствие чугуна': 15,
 'вн.пл.прост._Обрыв горловины': 16,
 'вн.пл.прост._Наложение продувки': 17,
 'межпл.прост._Отсутствие чугуна': 18,
 'вн.пл.прост._Вхождение в гр.МНЛЗ': 19,
 'межпл.прост._Полусухое торкрет.': 20,
 'межпл.прост._Обрыв горловины': 21,
 'вн.пл.прост._Отсутствие O2': 22,
 'опер_Охл. азотом': 23,
 'межпл.прост._Подварка  футеровки': 24,
 'межпл.прост._Вхождение в гр.МНЛЗ': 25,
 'вн.пл.прост._Неиспр. электр. обор': 26,
 'межпл.прост._Наложение продувки': 27,
 'вн.пл.прост._Ожидание стальковша': 28,
 'межпл.прост._Замер положения фурм': 29,


In [44]:
train_mean = train_texts.apply(lambda x: tokens2m(x, model_wv))
test_mean = test_texts.apply(lambda x: tokens2m(x, model_wv))

In [49]:
train_mean = pd.DataFrame.from_dict(dict(zip(train_mean.index, train_mean.values))).T
test_mean = pd.DataFrame.from_dict(dict(zip(test_mean.index, test_mean.values))).T

In [51]:
train_mean.columns = ["w2v_" + str(i) for i in range(20)]
test_mean.columns = ["w2v_" + str(i) for i in range(20)]

In [56]:
train_mean = train_mean.reset_index().rename(columns={"index": "NPLV"})
test_mean = test_mean.reset_index().rename(columns={"index": "NPLV"})

In [57]:
train_mean.to_csv("train_mean.csv", index=False)
test_mean.to_csv("test_mean.csv", index=False)

In [58]:
train_mean

,NPLV,w2v_0,w2v_1,w2v_2,w2v_3,w2v_4,w2v_5,w2v_6,w2v_7,w2v_8,w2v_9,w2v_10,w2v_11,w2v_12,w2v_13,w2v_14,w2v_15,w2v_16,w2v_17,w2v_18,w2v_19
0,510008,-0.114882,0.030544,0.044728,-0.090177,0.088356,-0.037197,0.057584,0.377415,-0.186661,0.140914,0.147470,-0.089297,-0.022813,-0.077599,0.095621,0.045854,0.439980,-0.045212,-0.273906,-0.021222
1,510009,-0.110043,0.029952,0.065641,-0.081639,0.061863,-0.046468,0.098110,0.375402,-0.193014,0.114485,0.178080,-0.087972,-0.044603,-0.098594,0.096370,0.050012,0.431295,-0.012977,-0.253913,-0.056444
2,510010,-0.121507,0.041272,0.039692,-0.082083,0.087226,-0.037164,0.050566,0.353338,-0.179296,0.136604,0.132843,-0.088877,-0.019686,-0.071058,0.080427,0.039254,0.430053,-0.040293,-0.281537,-0.012427
3,510011,-0.114973,0.028978,0.015841,-0.073007,0.073870,-0.048484,0.029960,0.358324,-0.181542,0.121809,0.122959,-0.055501,-0.008472,-0.083334,0.075529,0.035109,0.426147,-0.072851,-0.308883,0.008847
4,510012,-0.126578,0.048984,0.024898,-0.100305,0.104101,-0.021002,0.023784,0.356717,-0.173509,0.155268,0.108573,-0.076031,-0.016428,-0.064929,0.065731,0.030755,0.429404,-0.052926,-0.288047,0.014150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2058,512318,-0.123621,0.043089,0.013794,-0.111800,0.114687,-0.015870,-0.011609,0.368293,-0.168578,0.169618,0.106456,-0.047371,-0.031029,-0.065103,0.065665,0.045875,0.421090,-0.083472,-0.304179,0.045313
2059,512319,-0.159826,0.046506,-0.004908,-0.094858,0.104034,-0.038849,0.042127,0.322277,-0.148469,0.142933,0.097297,-0.085416,0.002192,-0.069895,0.114603,0.048963,0.440689,-0.038802,-0.267387,-0.028631
2060,512320,-0.147689,0.053943,0.024311,-0.085061,0.077936,-0.030310,0.049240,0.351470,-0.174535,0.141803,0.100876,-0.092316,-0.029004,-0.058470,0.087236,0.024212,0.396489,-0.056873,-0.294153,0.011517
2061,512321,-0.146122,0.056021,0.019610,-0.090562,0.104351,-0.022772,0.039071,0.328681,-0.160331,0.145634,0.111100,-0.087943,-0.033430,-0.071396,0.077457,0.037544,0.422600,-0.031162,-0.275065,-0.000386


In [66]:
sip_train = pd.read_csv("../data/sip_train.csv")
sip_test = pd.read_csv("../data/sip_test.csv")

In [67]:
sip_train

,NPLV,VDSYP,NMSYP,VSSYP,DAT_OTD
0,510008,346,Уголь ТО,570,2021-01-01 03:03:53
1,510008,346,Уголь ТО,220,2021-01-01 03:04:10
2,510008,408,изв_ЦОИ,7300,2021-01-01 03:08:17
3,510008,346,Уголь ТО,270,2021-01-01 03:09:03
4,510008,346,Уголь ТО,430,2021-01-01 03:09:20
...,...,...,...,...,...
31579,512322,408,изв_ЦОИ,30,2021-04-26 18:28:06
31580,512322,408,изв_ЦОИ,490,2021-04-26 18:28:07
31581,512322,408,изв_ЦОИ,20,2021-04-26 18:28:22
31582,512322,408,изв_ЦОИ,970,2021-04-26 18:30:26


In [69]:
sip_train["concat"] = sip_train["VDSYP"].astype(str) + "_" + sip_train["NMSYP"] 
sip_test["concat"] = sip_test["VDSYP"].astype(str) + "_" + sip_test["NMSYP"] 

In [73]:
def tokens2m(tokens, model_wv):
    res = []
    for token in tokens:
        idx = model_wv.wv.key_to_index[token]
        res.append(model_wv.wv[idx])
        
    return np.mean(np.array(res), axis=0).tolist() + np.std(np.array(res), axis=0).tolist() + \
        [np.linalg.norm(np.array(res))]

vector_size = 10

train_texts = sip_train.groupby("NPLV")["concat"].apply(list)
test_texts = sip_test.groupby("NPLV")["concat"].apply(list)

texts_for_w2v = pd.concat([train_texts, test_texts])

model_wv = Word2Vec(min_count=1, vector_size=vector_size)

model_wv.build_vocab(list(texts_for_w2v), progress_per=200)

model_wv.train(texts_for_w2v, total_examples = model_wv.corpus_count, 
            epochs=10, report_delay=1)
model_wv.init_sims(replace=True)

train_mean = train_texts.apply(lambda x: tokens2m(x, model_wv))
test_mean = test_texts.apply(lambda x: tokens2m(x, model_wv))

train_mean = pd.DataFrame.from_dict(dict(zip(train_mean.index, train_mean.values))).T
test_mean = pd.DataFrame.from_dict(dict(zip(test_mean.index, test_mean.values))).T

train_mean.columns = ["w2v_" + str(i) for i in range(2 * vector_size + 1)]
test_mean.columns = ["w2v_" + str(i) for i in range(2 * vector_size + 1)]

train_mean = train_mean.reset_index().rename(columns={"index": "NPLV"}).set_index("NPLV")
test_mean = test_mean.reset_index().rename(columns={"index": "NPLV"}).set_index("NPLV")

<ipython-input-73-31382c5a6f79>:23: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model_wv.init_sims(replace=True)


In [75]:
lom_train = pd.read_csv("../data/lom_train.csv")
lom_test = pd.read_csv("../data/lom_test.csv")

In [76]:
lom_train

,NPLV,VDL,NML,VES
0,510008,4,К,56500
1,510008,8,О,16700
2,510008,13,КП,3000
3,510009,4,К,49800
4,510009,8,О,22800
...,...,...,...,...
6371,512321,8,О,4200
6372,512321,13,КП,3000
6373,512322,4,К,66700
6374,512322,13,КП,6000
